In [3]:
#먼저 이걸 돌려야 함. 길이 N >=1000 인 데이터만 모았음, 상장폐지 꺼졍
#google용으로 만들 때는 아래 gpath 주석 제거.
import pandas as pd
import glob
today_date='2020.04.29'
path1='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/dataset'
#gpath='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/dataset/'
gpath='/content/gdrive/Shared drives/STOCK/version1/dataset/'
available_path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/available/available.csv'
#############################삼성만큼 길이=5996 이지만 hard coding 을 피해보자.
f=pd.read_csv(open( path1+'/005930.csv' ))
N=f.shape[0]
#############################
files=glob.glob(path1+'/*.csv')
available=[]
for file in files:
    f=pd.read_csv(open(file.replace('\\', '/')))
    if f.shape[0]>=N:
        if f.loc[0,'날짜']==today_date:
            available.append(gpath+file.replace('\\', '/')[-10:-4]+'.csv')
print('Total :',len(files),' Available :',len(available),' percentage :',100*len(available)/len(files),'%')
av=pd.Series(available)
av.to_csv(available_path)

Total : 3605  Available : 295  percentage : 8.183079056865465 %


In [7]:
######################################어느정도까지 잘라야 하나 보자.
available=list(pd.read_csv(open('C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/available/available.csv')).iloc[:,1])
lengths={}
for file in available:
    f=pd.read_csv(open(file))
    temp=f.shape[0]
    lengths.setdefault(temp,[]).append(file)
M=max(lengths.keys())
print("maximum length :",M,"  maximum length indexs",len(lengths[M]))

maximum length : 5996   maximum length indexs 295


1.모델 러닝
=========

In [21]:
#모든 주식에 대해 돌려야 하는 코드
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import matplotlib.pyplot as plt
########################################################################################
path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/dataset/'
save_path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/models/'
trustval_path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/trustvalue/'
available_path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/available/available.csv'
seq_length=50
TARGET=2  #0: 종가 1: 시가 2: 고가 3: 저가
########################################################################################
def datapreparation(KEY_path):
    data=pd.read_csv(open(KEY_path))
    if int(data.loc[0,'거래량'])==0:          #FaceDivision 처리할 때 index out of range 를 방지하기 위함.
        data.loc[0,'거래량']=1
    if int(data.loc[data.shape[0]-1,'거래량'])==0:
        data.loc[data.shape[0]-1,'거래량']=1
    data.drop(['Unnamed: 0','전일비','날짜'],axis=1,inplace=True)
    data=data.reindex(index=data.index[::-1])

    return data

def FaceDivision(df):
    if np.isnan(df.to_numpy()).any():
        print("there exist NaN in raw data")
        print(df[df==np.nan])
        raise NotImplementedError
    if df.loc[0,'시가']==0:            #거래정지처리
        df.loc[0,'시가']=df.loc[0,'종가']
    temp1=pd.DataFrame(df.to_numpy(),columns=df.columns)
    temp2=temp1.drop('거래량',axis=1)
    index1=list(temp1[temp1['거래량']==0].index) 
    #비인기주는 아무도 관심을 주지 않아(...) 거래량이 0인 경우도 있다.
    #또한 액면분할은 여러번 있을 수 있다.
    
    idx=[]
    for i in range(len(index1)-1):
        if index1[i+1]-index1[i]>1:
            idx.append(int(i))
    idx.insert(0,-1)
    DivisionDays=[]
    for i in range(len(idx)-1):
        DivisionDays.append(index1[idx[i]+1:idx[i+1]+1])
    DivisionDays.append(index1[idx[-1]+1:])
    if len(DivisionDays[0])>0:
        for i in range(len(DivisionDays)):
            beforeDivision=temp2.loc[DivisionDays[i][0]-1,'종가']
            afterDivision=temp2.loc[DivisionDays[i][-1]+1,'시가']
            ratio=afterDivision/beforeDivision
            if 1/ratio>=1.5:                       #after 에 비해 before 가 1.5 배 이상이면 액면분할이라 가정
                temp2.loc[:DivisionDays[i][-1]]=temp2.loc[:DivisionDays[i][-1]]*ratio   #df의 slicing 은 거기까지 포함한다.
    
    return temp2

def Emptyfill(df):            #종가가 모두 존재한다는 가정 하에 시가/고가/저가 항목이 비어있으면 채워준다.
    temp=np.array(df.loc[:,'종가'])
    temp=np.where(temp==0,np.nan,temp)
    assert ~np.isnan(temp).any(), "there exist 0 in '종가' column"
    
    for item in ['시가','고가','저가']:
        index1=list(df[df[item].map(int)==0].index)
        for each in index1:
            df.loc[each,item]=df.loc[each,'종가']
    
    return df

def df2nps(df,seqlen):
    seq_len=seqlen+1
    temp=df.to_numpy()
    result=[]
    for i in range(df.shape[0]-seq_len):
        result.append( temp[i:i+seq_len,:] )
        
    return np.array(result)

def normalization_and_split(result,train_ratio,TARGET):
    denoms=[]
    for table in result:
        temp=[]
        for i in range(table.shape[1]):
            denominator=float(table[0,i])
            table[:,i]=(table[:,i]/denominator)-1
            temp.append(denominator)
        denoms.append(temp)
    denoms=np.array(denoms)

    #sklearn 에서 자동으로 섞어주는게 있는데 그것보단 순서 고려해서 쪼개는게 좋은듯
    ratio=int(round(len(result)*train_ratio))
    train=result[:ratio,:,:]
    np.random.shuffle(train)
    x_train=train[:,:-1,:]
    y_train=train[:,-1,TARGET]
    
    x_test=result[ratio:,:-1,:]
    y_test=result[ratio:,-1,TARGET]
    denoms=denoms[ratio:,TARGET]

    return x_train, y_train, x_test, y_test, denoms

def trustValue(model,history,x_test,y_test,denoms):
    pred = model.predict(x_test)
    Ytest=(y_test+1)*denoms
    Ypred=(pred.reshape(pred.shape[0])+1)*denoms
    Ytest_pct=pd.Series(Ytest).pct_change().dropna()
    Ypred_pct=pd.Series(Ypred).pct_change().dropna()
    CORR=np.corrcoef(Ytest_pct.values,Ypred_pct.values)

    return CORR[0,1]
########################################################################################
KEY_paths=pd.read_csv(open(available_path)).drop('Unnamed: 0',axis=1)
avail=list(KEY_paths.values.reshape((KEY_paths.shape[0])))
fail=[]
for KEY_path in KEY_paths.values:
    try:
        print("Processing with index",KEY_path[0][-10:-4])
        avail.remove(KEY_path[0])
        data=datapreparation(KEY_path[0])
        data= FaceDivision(data)
        data=Emptyfill(data)
        print("Data Length :",data.shape[0])
        result=df2nps( data, seq_length )
        x_train, y_train, x_test, y_test, denoms = normalization_and_split(result,0.9,TARGET)

        model=Sequential()  #LSTM, adam 이 그나마 나음.
        model.add( layers.LSTM(50, return_sequences=True,input_shape=(50,4)) )
        model.add( layers.LSTM(64, return_sequences=False))
        model.add( layers.Dense(1, activation='linear'))
        model.compile(loss='MSE',optimizer='adam',metrics=['accuracy'])

        history=model.fit(x_train,y_train,epochs=10,batch_size=10,verbose=1)   #여기에서 시간이 오래걸림
        if history.history[ 'loss' ][-1]<0.001:
            model.fit(x_train,y_train,epochs=10,batch_size=10,verbose=1)
            model.save(save_path+KEY_path[0][-10:-4]+'.h5')
            T=trustValue(model,x_test,y_test,denoms)
            print("Correlation Coefficient :",T)
            f = open(trustval_path+KEY_path[0][-10:-4]+'.txt', "w")
            f.write(str(T))
            f.close()
        else:
            print('loss is not converging. pass')
    except KeyboardInterrupt:
        avail.insert(0,KEY_path[0])
        av=pd.Series(avail)
        av.to_csv(available_path)
        ff=pd.Series(fail)
        ff.to_csv(available_path[:-13]+'ErrorLog.csv')
        import sys 
        sys.exit(0)
    except:
        print('TypeError')
        fail.append(KEY_path[0])
        pass

Processing with index 000180
Data Length : 5996
Train on 5350 samples
Epoch 1/20
  10/5350 [..............................] - ETA: 13:33

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:168: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



None
Traceback (most recent call last):
  File "<ipython-input-21-8948334d146a>", line 132, in <module>
    history=model.fit(x_train,y_train,epochs=20,batch_size=10,verbose=1)   #여기에서 시간이 오래걸림
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 342, in fit
    total_epochs=epochs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py", line 128, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\keras\engine\training_v2_utils.py", line 98, in execution_function
    distributed_function(input_fn))
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\eager\def_function.py", line 568, in __call__
    result = self._call

SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


2.모델로 다음날 예측
=======

In [ ]:
#모델을 불러오고 다음 값을 얻어내는 코드
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import matplotlib.pyplot as plt
########################################################################################
path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/dataset/'
save_path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/models/'
trustval_path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/trustvalue/'
seq_length=50
TARGET=2  #0: 종가 1: 시가 2: 고가 3: 저가  -  모델이 목표로 하는 값
Today_TARGET=0  #0: 종가 1: 시가 2: 고가 3: 저가  -  내일 TARGET 과 비교해서 수익률을 계산할 오늘의 값
########################################################################################
def datapreparation(KEY_path):
    data=pd.read_csv(open(KEY_path))
    if int(data.loc[0,'거래량'])==0:          #FaceDivision 처리할 때 index out of range 를 방지하기 위함.
        data.loc[0,'거래량']=1
    if int(data.loc[data.shape[0]-1,'거래량'])==0:
        data.loc[data.shape[0]-1,'거래량']=1
    data.drop(['Unnamed: 0','전일비','날짜'],axis=1,inplace=True)
    data=data.reindex(index=data.index[::-1])

    return data

def FaceDivision(df):
    if np.isnan(df.to_numpy()).any():
        print("there exist NaN in raw data")
        print(df[df==np.nan])
        raise NotImplementedError
    if df.loc[0,'시가']==0:            #거래정지처리
        df.loc[0,'시가']=df.loc[0,'종가']
    temp1=pd.DataFrame(df.to_numpy(),columns=df.columns)
    temp2=temp1.drop('거래량',axis=1)
    index1=list(temp1[temp1['거래량']==0].index) 
    #비인기주는 아무도 관심을 주지 않아(...) 거래량이 0인 경우도 있다.
    #또한 액면분할은 여러번 있을 수 있다.
    
    idx=[]
    for i in range(len(index1)-1):
        if index1[i+1]-index1[i]>1:
            idx.append(int(i))
    idx.insert(0,-1)
    DivisionDays=[]
    for i in range(len(idx)-1):
        DivisionDays.append(index1[idx[i]+1:idx[i+1]+1])
    DivisionDays.append(index1[idx[-1]+1:])
    if len(DivisionDays[0])>0:
        for i in range(len(DivisionDays)):
            beforeDivision=temp2.loc[DivisionDays[i][0]-1,'종가']
            afterDivision=temp2.loc[DivisionDays[i][-1]+1,'시가']
            ratio=afterDivision/beforeDivision
            if 1/ratio>=1.5:                       #after 에 비해 before 가 1.5 배 이상이면 액면분할이라 가정
                temp2.loc[:DivisionDays[i][-1]]=temp2.loc[:DivisionDays[i][-1]]*ratio   #df의 slicing 은 거기까지 포함한다.
    
    return temp2

def Emptyfill(df):            #종가가 모두 존재한다는 가정 하에 시가/고가/저가 항목이 비어있으면 채워준다.
    temp=np.array(df.loc[:,'종가'])
    temp=np.where(temp==0,np.nan,temp)
    assert ~np.isnan(temp).any(), "there exist 0 in '종가' column"
    
    for item in ['시가','고가','저가']:
        index1=list(df[df[item].map(int)==0].index)
        for each in index1:
            df.loc[each,item]=df.loc[each,'종가']
    
    return df

def Prediction(code):
    ###########################################################ImportModel
    model=tf.keras.models.load_model(save_path+code+'.h5')
    ###########################################################ImportTrustValue
    T=float(  open(trustval_path+code+'.txt').readline()  )
    ###########################################################
    data=Emptyfill( FaceDivision( datapreparation(path+code+'.csv') ) )
    data=data.iloc[data.shape[0]-seq_length:,:].to_numpy()
    Today=list(data[-1,:])
    temp=[]
    for i in range(data.shape[1]):
        denominator=float(data[0,i])
        data[:,i]=(data[:,i]/denominator)-1
        temp.append(denominator)
    ###########################################################
    data=data.reshape((1,seq_length,4))
    y=model.predict(data)
    TomorrowHigh=(y+1)*temp[TARGET]
    r=float(TomorrowHigh-Today[Today_TARGET])/float(Today[Today_TARGET])
    trusted=float(r*T)
    
    return Today,TomorrowHigh,r,trusted

import glob
files=glob.glob(save_path[:-1]+'/*.h5')
codeNtrusted={}
for codepath in files:
    try:
        code=codepath.replace('\\','/')[-9:-3]
        print('processing with code :',code)
        with tf.device('/device:GPU:0'):
            _,_,_,trusted=Prediction(code)
        codeNtrusted.setdefault(trusted,[]).append(code)
    except KeyboardInterrupt:
        import sys
        sys.exit(0)
    except:
        print('Prediction about index',code,'failed. Pass')
        pass

MaximumTrust=max(codeNtrusted.keys())
codes=codeNtrusted[MaximumTrust]
for code in codes:
    with tf.device('/device:GPU:0'):
    Today,TomorrowHigh,r,trusted=Prediction(code)
    print("code :",code,"Today :",Today,"Predicted Tomorrow High :",int(TomorrowHigh[0]),'Predicted benefit : {}'.format(r*100),'Corr :',trusted/r)

삼성 test

In [37]:
#모델을 불러오고 다음 값을 얻어내는 코드
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import matplotlib.pyplot as plt
########################################################################################
path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/dataset/'
save_path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/models/'
trustval_path='C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/trustvalue/'
seq_length=50
TARGET=2  #0: 종가 1: 시가 2: 고가 3: 저가  -  모델이 목표로 하는 값
Today_TARGET=0  #0: 종가 1: 시가 2: 고가 3: 저가  -  내일 TARGET 과 비교해서 수익률을 계산할 오늘의 값
########################################################################################
def datapreparation(KEY_path):
    data=pd.read_csv(open(KEY_path))
    if int(data.loc[0,'거래량'])==0:          #FaceDivision 처리할 때 index out of range 를 방지하기 위함.
        data.loc[0,'거래량']=1
    if int(data.loc[data.shape[0]-1,'거래량'])==0:
        data.loc[data.shape[0]-1,'거래량']=1
    data.drop(['Unnamed: 0','전일비','날짜'],axis=1,inplace=True)
    data=data.reindex(index=data.index[::-1])

    return data

def FaceDivision(df):
    if np.isnan(df.to_numpy()).any():
        print("there exist NaN in raw data")
        print(df[df==np.nan])
        raise NotImplementedError
    if df.loc[0,'시가']==0:            #거래정지처리
        df.loc[0,'시가']=df.loc[0,'종가']
    temp1=pd.DataFrame(df.to_numpy(),columns=df.columns)
    temp2=temp1.drop('거래량',axis=1)
    index1=list(temp1[temp1['거래량']==0].index) 
    #비인기주는 아무도 관심을 주지 않아(...) 거래량이 0인 경우도 있다.
    #또한 액면분할은 여러번 있을 수 있다.
    
    idx=[]
    for i in range(len(index1)-1):
        if index1[i+1]-index1[i]>1:
            idx.append(int(i))
    idx.insert(0,-1)
    DivisionDays=[]
    for i in range(len(idx)-1):
        DivisionDays.append(index1[idx[i]+1:idx[i+1]+1])
    DivisionDays.append(index1[idx[-1]+1:])
    if len(DivisionDays[0])>0:
        for i in range(len(DivisionDays)):
            beforeDivision=temp2.loc[DivisionDays[i][0]-1,'종가']
            afterDivision=temp2.loc[DivisionDays[i][-1]+1,'시가']
            ratio=afterDivision/beforeDivision
            if 1/ratio>=1.5:                       #after 에 비해 before 가 1.5 배 이상이면 액면분할이라 가정
                temp2.loc[:DivisionDays[i][-1]]=temp2.loc[:DivisionDays[i][-1]]*ratio   #df의 slicing 은 거기까지 포함한다.
    
    return temp2

def Emptyfill(df):            #종가가 모두 존재한다는 가정 하에 시가/고가/저가 항목이 비어있으면 채워준다.
    temp=np.array(df.loc[:,'종가'])
    temp=np.where(temp==0,np.nan,temp)
    assert ~np.isnan(temp).any(), "there exist 0 in '종가' column"
    
    for item in ['시가','고가','저가']:
        index1=list(df[df[item].map(int)==0].index)
        for each in index1:
            df.loc[each,item]=df.loc[each,'종가']
    
    return df

def Prediction(code):
    ###########################################################ImportModel
    model=tf.keras.models.load_model(save_path+code+'.h5')
    ###########################################################ImportTrustValue
    T=float(  open(trustval_path+code+'.txt').readline()  )
    ###########################################################
    data=Emptyfill( FaceDivision( datapreparation(path+code+'.csv') ) )
    data=data.iloc[data.shape[0]-seq_length:,:].to_numpy()
    Today=list(data[-1,:])
    temp=[]
    for i in range(data.shape[1]):
        denominator=float(data[0,i])
        data[:,i]=(data[:,i]/denominator)-1
        temp.append(denominator)
    ###########################################################
    data=data.reshape((1,seq_length,4))
    y=model.predict(data)
    TomorrowHigh=(y+1)*temp[TARGET]
    r=float(TomorrowHigh-Today[Today_TARGET])/float(Today[Today_TARGET])
    trusted=float(r*T)
    
    return Today,TomorrowHigh,r,trusted

files=['C:/Users/mod96/Desktop/HSH/StudyingMaterials/Project - datascience/models/005930.h5']
codeNtrusted={}
for codepath in files:
    try:
        code=codepath.replace('\\','/')[-9:-3]
        print('processing with code :',code)
        _,_,_,trusted=Prediction(code)
        codeNtrusted.setdefault(trusted,[]).append(code)
    except KeyboardInterrupt:
        import sys
        sys.exit(0)
    except:
        print('ERROR')
        pass

MaximumTrust=max(codeNtrusted.keys())
codes=codeNtrusted[MaximumTrust]
for code in codes:
    Today,TomorrowHigh,r,trusted=Prediction(code)
    print("code :",code,"Today :",Today,"Predicted Tomorrow High :",int(TomorrowHigh[0]),'Predicted benefit : {}'.format(r*100),'Corr :',trusted/r)

processing with code : 005930
code : 005930 Today : [50000.0, 49900.0, 50500.0, 49600.0] Predicted Tomorrow High : 50986 Predicted benefit : 1.972296875 Corr : 0.29932204
